In [1]:
import pandas as pd
import json

In [2]:
def save_to_jsonl(mtd, outfilename):
    with open(outfilename, 'a+', encoding='utf8') as outfile:
        #for entry in JSON_file:
        json.dump(mtd, outfile, ensure_ascii=False)
        outfile.write('\n')

In [3]:
###############
# Use this fnx to get all habitats for a given micro-organism
# Input : data -> relations.full pandas Dataframe
# Output : Jsonl file from save_to_jsonl() fnx, to be indexed in ES (logstash input)
#############
def process_beyond_db(data):
    longList = []
    data['orgdict']=data[['form_microorganism','taxid','pmid','paths_microorganism', 'offset_microorganism', 'form_habitat']].to_dict("records")
    data['microorganism'] = 'microorganism'
    df = data[['microorganism','orgdict']]
    df['microorganism'] = df['microorganism'].apply(str) #Converting integer column 'label' to string
    df = df.groupby('microorganism')['orgdict'].apply(list) 
    desired_dict = df.to_dict()
    
    data['habtdict']=data[['form_habitat','obtid','pmid','paths_habitat', 'offset_habitat']].to_dict("records")
    data['habitat'] = 'habitat'
    df = data[['habitat','habtdict']]
    df['habitat'] = df['habitat'].apply(str) #Converting integer column 'label' to string
    df = df.groupby('habitat')['habtdict'].apply(list) 
    desired_dict_hab = df.to_dict()
    
    d = (pd.DataFrame(desired_dict['microorganism'])
       .groupby(['form_microorganism'])
       .agg(set)
       .reset_index()
       .to_dict('r'))
    
    dhb = (pd.DataFrame(desired_dict_hab['habitat'])
       .groupby(['form_habitat'])
       .agg(set)
       .reset_index()
       .to_dict('r'))
    
    for v in d:
        v['habitats'] = []
        v['taxid'] = list(v['taxid'])[0]
    #     v['pmid'] = list(v['pmid'])
        v['pmid'] = [str(val) for val in list(v['pmid'])]
        v['paths_microorganism'] = list(v['paths_microorganism'])[0]
        v['offset_microorganism'] = list(v['offset_microorganism'])
        v['form_habitat'] = list(v['form_habitat'])
        
    for v in dhb:
    #     v['habitats'] = []
        v['obtid'] = list(v['obtid'])[0]
#         v['pmid'] = list(v['pmid'])
        v['pmid'] = [str(val) for val in list(v['pmid'])]
        v['paths_habitat'] = list(v['paths_habitat'])[0]
        v['offset_habitat'] = list(v['offset_habitat'])[0]
        
    for i in range(len(d)):
        for j in range(len(dhb)):
            if dhb[j]['form_habitat'] in d[i]['form_habitat']:
                d[i]['habitats'].append(dhb[j]) 
            #     print(dhb[0]['form_habitat'])
    
    # D['Microorganism'].append(d[0])
    for mi in d:
        D = {}
        D['Microorganism'] = []
        D['Microorganism'].append(mi)
        save_to_jsonl(D, 'beyond_db.jsonl')
#         longList.append(D)
#     return longList

In [4]:
###### 
#  Use this fnx to get each relation (miroorganism <-> habitats) with translated paths
# Build Losgstash input data for ES from these input data :
# 1 - relations file
# 2 - BioNLP_onto,
# 3 - taxid_microorganisms
#####
def beyond_db_preprocess(relations,BioNLP_onto,taxid_microorganisms):
#     relations = "/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/relations.full.tsv/relations_min.tsv"
    data = pd.read_csv(relations, delimiter='\t')
    df = data[['form_microorganism', 'lemma_microorganism', 'paths_microorganism','pmid', 'section', 'taxid',
        'canonical_microorganism', 
       'obtid', 'form_habitat', 'lemma_habitat', 'canonical_habitat',
       'paths_habitat', 'offset_microorganism', 'offset_habitat']]
    filtered_df = df[df['form_microorganism'].notnull()]
    
    filtered_df_dic = filtered_df.to_dict('records')
    val = list(map(dict, set(tuple(sorted(d.items())) for d in filtered_df_dic)))
    print(len(val))
    
    test = filtered_df[:5] # consider subset for test
    
    # define translated path --> ordered list of habitats
    test['translated_path_habitat'] = test['paths_habitat'].str.split("/")
        
    dc = {}
    synod = {}
    with open (BioNLP_onto, "r") as hfile:
        sp = hfile.read()
        lines = sp.split("\n\n")
        for line in lines:
            item = line.split('\n\n\n')
            items = [i.split('\n') for i in item]
            items = [x.split(': ') for x in items[0]]
            id_name = [l for l in items if l[0]=='id' or l[0]=='name']
            syno =  [l for l in items if l[0]=='id' or l[0]=='synonym']

            if len(id_name)>0:
                dc[id_name[0][1]] = id_name[1][1]
            #   print(syno)
            ########## We can add synonyms if needed
            if len(syno)>1:
                synod[syno[0][1]] = [ i[1] for i in syno[1:]]
            ############

    for k,v in dc.items():
        test['translated_path_habitat'] = test.apply(lambda x:[i.replace(k,v) for i in x['translated_path_habitat'] if i], axis=1)
        
    # pop current habitat from the path
    test['translated_path_habitat'] = test['translated_path_habitat'].apply(lambda x:x[:-1])
    
    
    ############Build translated path for each micro-organism
#     taxid_microorganisms = '/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/taxid_microorganisms.tsv/taxid_microorganisms.tsv'
    taxid_data = pd.read_csv(taxid_microorganisms, delimiter='\t')
    
    taxid_data = taxid_data[ ['taxid', 'taxon']]
    
    taxid_data_dic = taxid_data.set_index('taxid').T.to_dict('r')
    
    test['translated_paths_microorganism'] = test['paths_microorganism'].str.split("/")
    
    for k,v in taxid_data_dic[0].items():
        test['translated_paths_microorganism'] = test.apply(lambda x:[i.replace(k,v) for i in x['translated_paths_microorganism'] if i], axis=1)
    
    # pop current habitat from the path
    test['translated_paths_microorganism'] = test['translated_paths_microorganism'].apply(lambda x:x[:-1])

    test_dic = test.to_dict('records')    
    for d in test_dic:
        d['pmid'] = str(d['pmid'])
        
    ################### Save record!
    for mtd in test_dic:
        save_to_jsonl(mtd, 'beyond_db_uniq5.jsonl')

In [7]:
relations = "./data-examples/relations.full.tsv/relations_min.tsv"
BioNLP_onto = './data-examples/BioNLP-OST+EnovFood.txt'
taxid_microorganisms = './data-examples/taxid_microorganisms.tsv/taxid_microorganisms.tsv'

In [ ]:
beyond_db_preprocess(relations,BioNLP_onto,taxid_microorganisms)

/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


100309


/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 